In [1]:
import os
os.environ.setdefault('DJANGO_SETTINGS_MODULE', 'myzaka.settings')
import django
django.setup()

import urllib3
import requests
import pandas as pd

In [2]:
from app import models

from sqlalchemy import create_engine
engine = create_engine('sqlite:///myzakadb.sqlite3')

In [3]:
engine.table_names()

['app_audits',
 'app_balancesheet',
 'app_capital',
 'app_cashflow',
 'app_forecast',
 'app_incomeexpense',
 'app_maintenance',
 'app_municipalities',
 'app_municipalities_audits',
 'app_municipalities_balance_sheets',
 'app_municipalities_capital',
 'app_municipalities_cash_flows',
 'app_municipalities_forecasts',
 'app_municipalities_income_expense',
 'app_municipalities_maintenance',
 'app_municipalities_officials',
 'app_municipalities_wasteful',
 'app_officials',
 'app_wastefulexpenditure',
 'auth_group',
 'auth_group_permissions',
 'auth_permission',
 'auth_user',
 'auth_user_groups',
 'auth_user_user_permissions',
 'django_admin_log',
 'django_content_type',
 'django_migrations',
 'django_session',
 'sqlite_sequence']


### Generate Officials data

In [4]:
try:
    officials = pd.DataFrame(requests.get('https://municipaldata.treasury.gov.za/api/cubes/officials/facts',
                                          allow_redirects=True).json()['data'])

    officials.fillna('not availiable', inplace=True)
    for col in officials.columns:
        officials[col] = officials[col].apply(lambda x: str(x).lower())

    officials = officials[['contact_details.title',  'role.role', 'contact_details.name',  'contact_details.email_address', 
               'contact_details.fax_number', 'contact_details.phone_number', 'municipality.demarcation_code']]

    officials.rename(columns={'contact_details.title':'title',  'role.role':'role', 'contact_details.name':'name',  
                                                            'contact_details.email_address':'email_address', 'contact_details.fax_number':'fax_number', 
                                                             'contact_details.phone_number':'phone_number', 'municipality.demarcation_code':'demarcation_code'}, inplace=True)
    officials.drop_duplicates(inplace=True)

    models.Officials.objects.all().delete()
    officials.to_sql('app_officials', engine, if_exists='append', index=False)
    models.Officials.objects.count()
except Exception as ex:
    print("ERROR:", ex)

### Generate BalanceSheet data

In [5]:
try:
    balance_sheet = pd.DataFrame(requests.get('https://municipaldata.treasury.gov.za/api/cubes/bsheet/facts',
                                          allow_redirects=True).json()['data'])

    balance_sheet.amount = balance_sheet.amount.transform(lambda x: x.fillna('0'))

    for col in balance_sheet.columns:
        balance_sheet[col] = balance_sheet[col].apply(lambda x: str(x).lower())

    balance_sheet = balance_sheet[['demarcation.label', 'demarcation.code', 'financial_year_end.year', 'amount', 'item.label', 'amount_type.label']]
    balance_sheet.rename(columns={'demarcation.code':'demarcation_code', 'demarcation.label':'demarcation_label', 
                              'financial_year_end.year':'financial_year_end', 'item.label':'item_label', 
                              'amount':'amount', 'amount_type.label':'amount_type'}, inplace=True)
    balance_sheet.amount = balance_sheet.amount.transform(lambda x: float(x))
    balance_sheet.financial_year_end = balance_sheet.financial_year_end.transform(lambda x: int(x))

    balance_sheet = balance_sheet.sort_values(by=['financial_year_end', 'amount'], ascending=False)
    balance_sheet.drop_duplicates(inplace=True, subset=['demarcation_code', 
                                          'financial_year_end','item_label', 'amount_type'])

    balance_sheet.fillna('not availiable', inplace=True)
    models.BalanceSheet.objects.all().delete()
    balance_sheet.to_sql('app_balancesheet', engine, if_exists='append', index=False)
    models.BalanceSheet.objects.count()
    
except Exception as ex:
    print("ERROR:", ex)

### Generate Cash Flow data

In [6]:
try:
    cash_flows = pd.DataFrame(requests.get('https://municipaldata.treasury.gov.za/api/cubes/cflow/facts',
                                          allow_redirects=True).json()['data'])

    cash_flows.amount = cash_flows.amount.transform(lambda x: x.fillna('0'))

    for col in cash_flows.columns:
        cash_flows[col] = cash_flows[col].apply(lambda x: str(x).lower())

    cash_flows_df = cash_flows[['amount_type.label', 'demarcation.code', 'financial_year_end.year', 'amount', 'item.label']]
    cash_flows_df.rename(columns={'demarcation.code':'demarcation_code', 'amount_type.label':'amount_type', 
                              'financial_year_end.year':'financial_year_end', 'item.label':'item_label'}, inplace=True)
    cash_flows_df.amount = cash_flows_df.amount.transform(lambda x: float(x))
    cash_flows_df.financial_year_end = cash_flows_df.financial_year_end.transform(lambda x: int(x))

    cash_flows_df = cash_flows_df.sort_values(by=['financial_year_end', 'amount'], ascending=False)
    cash_flows_df.drop_duplicates(inplace=True, subset=['demarcation_code', 'amount_type',
                                                       'financial_year_end', 'item_label'])

    cash_flows_df.fillna('not availiable', inplace=True)
    models.CashFlow.objects.all().delete()
    cash_flows_df.to_sql('app_cashflow', engine, if_exists='append', index=False)
    models.CashFlow.objects.count()
    
except Exception as ex:
    print("ERROR:", ex)

/home/doctor/anaconda3/lib/python3.6/site-packages/pandas/core/frame.py:2746: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  **kwargs)
/home/doctor/anaconda3/lib/python3.6/site-packages/pandas/core/generic.py:3110: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self[name] = value


### Generate Maintenance data

In [7]:
try:
    repairs = pd.DataFrame(requests.get('https://municipaldata.treasury.gov.za/api/cubes/repmaint/facts',
                                          allow_redirects=True).json()['data'])

    repairs.amount = repairs.amount.transform(lambda x: x.fillna('0'))
    for col in repairs.columns:
        repairs[col] = repairs[col].apply(lambda x: str(x).lower())

    repairs_df = repairs[['amount_type.label', 'demarcation.code', 'financial_year_end.year', 'amount', 'item.label']]

    repairs_df.rename(columns={'demarcation.code':'demarcation_code', 'amount_type.label':'amount_type', 
                              'financial_year_end.year':'financial_year_end', 'item.label':'item_label'}, inplace=True)
    repairs_df.amount = repairs_df.amount.transform(lambda x: float(x))
    repairs_df.financial_year_end = repairs_df.financial_year_end.transform(lambda x: int(x))

    repairs_df = repairs_df.sort_values(by=['financial_year_end', 'amount'], ascending=False)
    repairs_df.drop_duplicates(inplace=True, subset=['demarcation_code', 'amount_type',
                                                       'financial_year_end', 'item_label'])

    repairs_df.fillna('not availiable', inplace=True)
    models.Maintenance.objects.all().delete()
    repairs_df.to_sql('app_maintenance', engine, if_exists='append', index=False)
    models.Maintenance.objects.count()
    
except Exception as ex:
    print("ERROR:", ex)

/home/doctor/anaconda3/lib/python3.6/site-packages/pandas/core/frame.py:2746: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  **kwargs)
/home/doctor/anaconda3/lib/python3.6/site-packages/pandas/core/generic.py:3110: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self[name] = value


In [8]:
#repairs_df[(repairs_df['demarcation_code']=='buf') & (repairs_df['financial_year_end'] == '2012') & (repairs_df['item_label']=='contracted services')]

In [9]:
#repairs_df.groupby(['demarcation_code', 'financial_year_end', 'item_label']).count()

### Generate Audits data

In [10]:
try:
    audits = pd.DataFrame(requests.get('https://municipaldata.treasury.gov.za/api/cubes/audit_opinions/facts',
                                          allow_redirects=True).json()['data'])

    audits.fillna('not availiable', inplace=True)
    for col in audits.columns:
        audits[col] = audits[col].apply(lambda x: str(x).lower())

    audits_df = audits[['demarcation.code', 'demarcation.label', 'financial_year_end.year', 'opinion.label', 'opinion.report_url']]
    audits_df.rename(columns={'demarcation.code':'demarcation_code', 'demarcation.label':'demarcation_label', 
                              'financial_year_end.year':'financial_year_end', 'opinion.label':'opinion', 
                              'opinion.report_url':'opinion_report_url'}, inplace=True)

    audits_df.financial_year_end = audits_df.financial_year_end.transform(lambda x: int(x))

    audits_df = audits_df.sort_values(by=['financial_year_end'], ascending=False)
    audits_df.drop_duplicates(inplace=True) 

    models.Audits.objects.all().delete()
    audits_df.to_sql('app_audits', engine, if_exists='append', index=False)
    print("Object Count:", models.Audits.objects.count())
except Exception as ex:
    print("ERROR:", ex)

Object Count: 1632


/home/doctor/anaconda3/lib/python3.6/site-packages/pandas/core/frame.py:2746: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  **kwargs)
/home/doctor/anaconda3/lib/python3.6/site-packages/pandas/core/generic.py:3110: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self[name] = value


### Generate WastefulExpese data

In [11]:
try:
    wasteful = pd.DataFrame(requests.get('https://municipaldata.treasury.gov.za/api/cubes/uifwexp/facts',
                                          allow_redirects=True).json()['data'])

    wasteful.amount = wasteful.amount.transform(lambda x: x.fillna(0))

    for col in wasteful.columns:
        wasteful[col] = wasteful[col].apply(lambda x: str(x).lower())

    wasteful.drop(['item.code'], axis=1, inplace=True)
    wasteful.rename(columns={'demarcation.code':'demarcation_code', 'demarcation.label':'demarcation_label', 
                            'financial_year_end.year':'financial_year_end', 'item.label':'item_label'}, inplace=True)
    wasteful.amount = wasteful.amount.transform(lambda x: float(x))
    wasteful.financial_year_end = wasteful.financial_year_end.transform(lambda x: int(x))

    wasteful = wasteful.sort_values(by=['financial_year_end', 'amount'], ascending=False)
    wasteful.drop_duplicates(inplace=True)

    wasteful.fillna('not availiable', inplace=True)
    models.WastefulExpenditure.objects.all().delete()
    wasteful.to_sql('app_wastefulexpenditure', engine, if_exists='append', index=False)
    print("Object Count:", models.WastefulExpenditure.objects.count())
    
except Exception as ex:
    print("ERROR:", ex)

Object Count: 3333


### Generate Capital Data

In [12]:
try:
    munis = pd.DataFrame(list(models.Officials.objects.all().values()))
    muni_codes = [str(x).upper() for x in pd.unique(munis.demarcation_code)]
    models.Capital.objects.all().delete()
    count = 1
    for code in muni_codes:
        print("processing {0} of {1}, {2} remaining...".format(count, len(muni_codes), len(muni_codes) - count))
        capital = pd.DataFrame(requests.get('https://municipaldata.treasury.gov.za/api/cubes/capital/aggregate?drilldown=demarcation.code|demarcation.label|item.label|financial_year_end.year|amount_type.label&cut=demarcation.code%3A%22' + code + '%22&aggregates=new_assets.sum|renewal_of_existing.sum|total_assets.sum',
                                          allow_redirects=True).json()['cells'])

        capital = capital[['amount_type.label', 'financial_year_end.year', 'item.label', 
                           'demarcation.code', 'demarcation.label', 'renewal_of_existing.sum', 
                           'total_assets.sum', 'new_assets.sum']]
        capital.rename(columns={'amount_type.label':'amount_type', 
                                'new_assets.sum':'sum_new_assets',
                                'financial_year_end.year':'financial_year_end', 
                                'renewal_of_existing.sum':'sum_renewed_assets', 
                                'item.label':'item_label', 
                                'total_assets.sum':'sum_total_assets',
                                'demarcation.code':'demarcation_code', 
                                'demarcation.label':'demarcation_label'}, inplace=True)

        capital.sum_new_assets = capital.sum_new_assets.transform(lambda x: x.fillna(0))
        capital.sum_renewed_assets = capital.sum_renewed_assets.transform(lambda x: x.fillna(0))
        capital.sum_total_assets = capital.sum_total_assets.transform(lambda x: x.fillna(0))

        for col in capital.columns:
            capital[col] = capital[col].apply(lambda x: str(x).lower())

        capital.sum_new_assets = capital.sum_new_assets.transform(lambda x: float(x))
        capital.sum_renewed_assets = capital.sum_renewed_assets.transform(lambda x: float(x))
        capital.sum_total_assets = capital.sum_total_assets.transform(lambda x: float(x))
        capital.financial_year_end = capital.financial_year_end.transform(lambda x: int(x))

        capital = capital.sort_values(by=['financial_year_end', 'sum_total_assets', 
                                          'sum_renewed_assets', 'sum_new_assets'], ascending=False)
        capital.drop_duplicates(inplace=True)

        capital.fillna('not availiable', inplace=True)
        models.Capital.objects.all().delete()
        capital.to_sql('app_capital', engine, if_exists='append', index=False)
        models.Capital.objects.count()
        count += 1
        
except Exception as ex:
    print("ERROR:", ex)

processing 1 of 292, 291 remaining...
processing 2 of 292, 290 remaining...
processing 3 of 292, 289 remaining...
processing 4 of 292, 288 remaining...
processing 5 of 292, 287 remaining...
processing 6 of 292, 286 remaining...
processing 7 of 292, 285 remaining...
processing 8 of 292, 284 remaining...
processing 9 of 292, 283 remaining...
processing 10 of 292, 282 remaining...
processing 11 of 292, 281 remaining...
processing 12 of 292, 280 remaining...
processing 13 of 292, 279 remaining...
processing 14 of 292, 278 remaining...
processing 15 of 292, 277 remaining...
processing 16 of 292, 276 remaining...
processing 17 of 292, 275 remaining...
processing 18 of 292, 274 remaining...
processing 19 of 292, 273 remaining...
processing 20 of 292, 272 remaining...
processing 21 of 292, 271 remaining...
processing 22 of 292, 270 remaining...
processing 23 of 292, 269 remaining...
processing 24 of 292, 268 remaining...
processing 25 of 292, 267 remaining...
processing 26 of 292, 266 remainin

processing 209 of 292, 83 remaining...
processing 210 of 292, 82 remaining...
processing 211 of 292, 81 remaining...
processing 212 of 292, 80 remaining...
processing 213 of 292, 79 remaining...
processing 214 of 292, 78 remaining...
processing 215 of 292, 77 remaining...
processing 216 of 292, 76 remaining...
processing 217 of 292, 75 remaining...
processing 218 of 292, 74 remaining...
processing 219 of 292, 73 remaining...
processing 220 of 292, 72 remaining...
processing 221 of 292, 71 remaining...
processing 222 of 292, 70 remaining...
processing 223 of 292, 69 remaining...
processing 224 of 292, 68 remaining...
processing 225 of 292, 67 remaining...
processing 226 of 292, 66 remaining...
processing 227 of 292, 65 remaining...
processing 228 of 292, 64 remaining...
processing 229 of 292, 63 remaining...
processing 230 of 292, 62 remaining...
processing 231 of 292, 61 remaining...
processing 232 of 292, 60 remaining...
processing 233 of 292, 59 remaining...
processing 234 of 292, 58

### Generate IncomeExpense data

In [13]:
try:
    munis = pd.DataFrame(list(models.Officials.objects.all().values()))
    muni_codes = [str(x).upper() for x in pd.unique(munis.demarcation_code)]
    models.IncomeExpense.objects.all().delete()
    count = 1
    for code in muni_codes:
        print("processing {0} of {1}, {2} remaining...".format(count, len(muni_codes), len(muni_codes) - count))
        incexp = pd.DataFrame(
            requests.get('https://municipaldata.treasury.gov.za/api/cubes/incexp/aggregate?drilldown=amount_type.label|financial_year_end.year|demarcation.code|demarcation.label|item.label&cut=demarcation.code:%22'+ code +'%22&aggregates=amount.sum',
                                          allow_redirects=True).json()['cells']
        )

        incexp['amount.sum'] = incexp['amount.sum'].transform(lambda x: x.fillna(0))
        for col in incexp.columns:
            incexp[col] = incexp[col].apply(lambda x: str(x).lower())

        incexp = incexp[['amount_type.label', 'financial_year_end.year', 'item.label', 'demarcation.code', 'demarcation.label', 'amount.sum']]
        incexp.rename(columns={'amount_type.label':'amount_type',  
                               'financial_year_end.year':'financial_year_end', 
                               'amount.sum':'amount',
                                'item.label':'item_label', 'demarcation.code':'demarcation_code', 'demarcation.label':'demarcation_label'}, 
                      inplace=True)
        incexp.amount = incexp.amount.transform(lambda x: float(x))
        incexp.financial_year_end = incexp.financial_year_end.transform(lambda x: int(x))

        incexp = incexp.sort_values(by=['financial_year_end', 'amount'], ascending=False)
        incexp.drop_duplicates(inplace=True)

        incexp.fillna('not availiable', inplace=True)
        incexp.groupby('financial_year_end').count()
        incexp.to_sql('app_incomeexpense', engine, if_exists='append', index=False)
        print("Row Count", models.IncomeExpense.objects.count())
        count += 1
        
except Exception as ex:
    print("ERROR:", ex)

processing 1 of 292, 291 remaining...
Row Count 3015
processing 2 of 292, 290 remaining...
Row Count 6231
processing 3 of 292, 289 remaining...
Row Count 9380
processing 4 of 292, 288 remaining...
Row Count 12596
processing 5 of 292, 287 remaining...
Row Count 15410
processing 6 of 292, 286 remaining...
Row Count 18358
processing 7 of 292, 285 remaining...
Row Count 21373
processing 8 of 292, 284 remaining...
Row Count 23986
processing 9 of 292, 283 remaining...
Row Count 27202
processing 10 of 292, 282 remaining...
Row Count 30418
processing 11 of 292, 281 remaining...
Row Count 33500
processing 12 of 292, 280 remaining...
Row Count 36716
processing 13 of 292, 279 remaining...
Row Count 39798
processing 14 of 292, 278 remaining...
Row Count 43014
processing 15 of 292, 277 remaining...
Row Count 46230
processing 16 of 292, 276 remaining...
Row Count 49379
processing 17 of 292, 275 remaining...
Row Count 52528
processing 18 of 292, 274 remaining...
Row Count 55744
processing 19 of 292, 

Row Count 415867
processing 148 of 292, 144 remaining...
Row Count 418882
processing 149 of 292, 143 remaining...
Row Count 421964
processing 150 of 292, 142 remaining...
Row Count 425046
processing 151 of 292, 141 remaining...
Row Count 427391
processing 152 of 292, 140 remaining...
Row Count 429736
processing 153 of 292, 139 remaining...
Row Count 432684
processing 154 of 292, 138 remaining...
Row Count 433287
processing 155 of 292, 137 remaining...
Row Count 436503
processing 156 of 292, 136 remaining...
Row Count 439518
processing 157 of 292, 135 remaining...
Row Count 441863
processing 158 of 292, 134 remaining...
Row Count 445012
processing 159 of 292, 133 remaining...
Row Count 448228
processing 160 of 292, 132 remaining...
Row Count 451444
processing 161 of 292, 131 remaining...
Row Count 454660
processing 162 of 292, 130 remaining...
Row Count 457876
processing 163 of 292, 129 remaining...
Row Count 461025
processing 164 of 292, 128 remaining...
Row Count 464174
processing 165

### Generate Munis data

In [14]:
munis = pd.DataFrame(requests.get('https://municipaldata.treasury.gov.za/api/cubes/municipalities/facts',
                                      allow_redirects=True).json()['data'])

munis = munis[['area', 'municipality.category', 'municipality.demarcation_code', 
                    'municipality.fax_number', 'municipality.name', 'municipality.phone_number', 'municipality.postal_address_1',
                    'municipality.postal_address_2', 'municipality.postal_address_3', 'municipality.province_name',
                    'municipality.street_address_1', 'municipality.street_address_2', 'municipality.street_address_3', 'municipality.url']]

munis.rename(columns={'area':'area', 'municipality.category':'category', 'municipality.demarcation_code':'demarcation_code', 
                    'municipality.fax_number':'fax_number', 'municipality.name':'name', 'municipality.phone_number':'phone_number', 
                      'municipality.postal_address_1':'postal_address_1','municipality.postal_address_2':'postal_address_2', 
                      'municipality.postal_address_3':'postal_code', 'municipality.province_name':'province_name',
                    'municipality.street_address_1':'street_address_1', 'municipality.street_address_2':'street_address_2', 
                      'municipality.street_address_3':'street_address_3', 'municipality.url':'website'}, inplace=True)

munis.fillna('not availiable', inplace=True)
for col in munis.columns:
    munis[col] = munis[col].apply(lambda x: str(x).lower())

models.Municipalities.objects.all().delete()

for muni in munis.values:
    dermancation_code = muni[2]
    officials = models.Officials.objects.filter( demarcation_code=dermancation_code)
    audits = models.Audits.objects.filter( demarcation_code=dermancation_code)
    balance_sheets = models.BalanceSheet.objects.filter( demarcation_code=dermancation_code)
    maintenance = models.Maintenance.objects.filter( demarcation_code=dermancation_code)
    cash_flow = models.CashFlow.objects.filter( demarcation_code=dermancation_code)
    capital = models.Capital.objects.filter( demarcation_code=dermancation_code)
    income_expense = models.IncomeExpense.objects.filter( demarcation_code=dermancation_code)
    forecast = models.Forecast.objects.filter( demarcation_code=dermancation_code)
    waste = models.WastefulExpenditure.objects.filter( demarcation_code=dermancation_code)
    
    muni = models.Municipalities.objects.create(
        area=muni[0],
        category=muni[1],
        demarcation_code=muni[2],
        fax_number=muni[3],
        name=muni[4],
        phone_number=muni[5],
        postal_address_1=muni[6],
        postal_address_2=muni[7],
        postal_code=muni[8],
        province_name=muni[9],
        street_address_1=muni[10],
        street_address_2=muni[11],
        street_address_3=muni[12],
        website=muni[13]
    )
    muni.officials.set(officials)
    muni.audits.set(audits)
    muni.balance_sheets.set(balance_sheets)
    muni.maintenance.set(maintenance)
    muni.cash_flows.set(cash_flow)
    muni.capital.set(capital)
    muni.income_expense.set(income_expense)
    muni.forecasts.set(forecast)
    muni.wasteful.set(waste)
    muni.save()

models.Municipalities.objects.count()

OperationalError: too many SQL variables